## 1 Introduction
### 1.1 Who's the target audience
This example is intended for audience who are comfortable with developing models using Jupyter notebook. A basic understanding of Azure Machine Learning (Azure ML) experiment is helpful. The purpose here is to give a complete walkthrough on using Jupyter notebook within Azure Machine Studio. Using linear regression as an example, we'll have the opportunity to see how Azure ML's Jupyter notebook can be used to fit a model, set up a web service on Azure ML and consume the service.

If you are new to Jupyter notebook, you can learn more about it from [The IPython Notebook][ipython link] and [Jupyter/IPython Notebook Quick Start Guide][jupyter link]. 

If you are new to Azure ML and want to learn more about it, the [Data Scientists' Guide][guide link] can help you get started.

[ipython link]: http://ipython.org/notebook.html
[jupyter link]: http://jupyter-notebook-beginner-guide.readthedocs.org/en/latest/index.html
[guide link]: https://gallery.cortanaanalytics.com/Experiment/Tutorial-for-Data-Scientists-3

### 1.2 Why Azure ML notebook
The major advantage of using Azure ML notebook is that you won't need to install Python on your local computer. This makes it possible for anyone with internet access to write Python programs using a web browser. Azure ML's notebook is based on the [Anaconda][Anaconda link] distribution, which has many packages installed. 

[Anaconda link]: https://www.continuum.io/

## 2 Data
In this example we'll be using the Boston housing dataset. There are 506 rows in the dataset. The target variable is median home price. There are 13 predictor variables including average number of rooms per dwelling, crime rate by town, etc. More information about this dataset can be found at [UCI][uci link].
[uci link]: https://archive.ics.uci.edu/ml/datasets/Housing

In [1]:
from sklearn.datasets import load_boston
boston = load_boston()
X = boston.data
y = boston.target
feature_names = boston.feature_names
print(X.shape)
print(y.shape)
print(feature_names)

(506, 13)
(506,)
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT' 'MEDV']


## 3 Linear regression with sklearn
Now let's assign the data to a new variable "mydata" and use it to develop a linear model. In practice, you may want to spend more time on things like feature engineering and variable selection. In this example, however, we will just fit a model using all variables. 

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# initiate the linear model and fit with data
lm = LinearRegression()
lm.fit(X, y)

# print the R-squared
print("The R-squared value is: {0:0.4f} \n".format(lm.score(X, y)))

# save intercept and coefficients
param_df = pd.DataFrame({"Features": ['intercept'] + list(feature_names), "Coef": [lm.intercept_] + list(lm.coef_)})
cols = param_df.columns.tolist()
cols = cols[-1:]+cols[:-1]
param_df = param_df[cols]
print(param_df)

The R-squared value is: 0.7406 

     Features       Coef
0   intercept  36.491103
1        CRIM  -0.107171
2          ZN   0.046395
3       INDUS   0.020860
4        CHAS   2.688561
5         NOX -17.795759
6          RM   3.804752
7         AGE   0.000751
8         DIS  -1.475759
9         RAD   0.305655
10        TAX  -0.012329
11    PTRATIO  -0.953464
12          B   0.009393
13      LSTAT  -0.525467


Next we'll use the model to make predictions. Typically, predictions are done on a validation dataset. Here, however, the training dataset is used for illustraton purposes.

In [3]:
newX = X
newY = y

# predict and create a dataframe
predicted = lm.predict(newX)
predictedDf = pd.DataFrame({"predicted": predicted})
# dataframe for X
newXDf = pd.DataFrame(newX)
newXDf.columns = feature_names
# dataframe for y
newYDf = pd.DataFrame(newY)
newYDf.columns = ['MEDV']
# join X, y and prediction
mydata_with_pd = newXDf.join(newYDf).join(predictedDf)
mydata_with_pd.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,predicted
0,0.00632,18,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,30.008213
1,0.02731,0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,25.029861
2,0.02729,0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,30.570232
3,0.03237,0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,28.608141
4,0.06905,0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,27.942882


With the predicted values, we can calculate metrics such as mean absolute error, root mean squared error, relative absolute error, and relative squared error. 

In [4]:
import numpy as np
obs = mydata_with_pd.MEDV
pred = mydata_with_pd.predicted

mae = np.mean(abs(pred-obs))
rmse = np.sqrt(np.mean((pred-obs)**2))
rae = np.mean(abs(pred-obs))/np.mean(abs(obs-np.mean(obs)))
rse = np.mean((pred-obs)**2)/np.mean((obs-np.mean(obs))**2)

print("Mean Absolute Error: {0:0.6f}".format(mae))
print("Root Mean Squared Error: {0:0.6f}".format(rmse))
print("Relative Absolute Error: {0:0.6f}".format(rae))
print("Relative Squared Error: {0:0.6f}".format(rse))

Mean Absolute Error: 3.272945
Root Mean Squared Error: 4.679506
Relative Absolute Error: 0.492379
Relative Squared Error: 0.259392


## 4 Web service

### 4.1 Set up a web service
After developing the model, we can deploy it as a service so others can use it. The "azureml" package's services subpackage can be used for this purpose. The following lines of code are setting up a web service named "demoservice".

In [5]:
# extract workspace info
from azureml import Workspace
ws = Workspace()
workdspace_id = ws.workspace_id
authorization_token = ws.authorization_token

# set up web service
from azureml import services
@services.publish(workdspace_id, authorization_token)
@services.types(crim=float, zn=float, indus=float, chas=float, nox=float, rm=float, age=float, 
                dis=float, rad=float, tax=float, ptratio=float, black=float, lstat=float)
@services.returns(float)
def demoservice(crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, black, lstat):
    # predict the label
    feature_vector = [crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, black, lstat]
    return lm.predict(feature_vector)

### 4.2 Consume a web service
After running the above code to set up a web service, we can consume it like the following example. This way of consuming the web service works only during the current session.

In [6]:
demoservice(0.00632, 18, 2.31, 0, 0.538, 6.575, 65.2, 4.09, 1, 296, 15.3, 396.9, 4.98)

30.008212692344642

To consume the web service out of the current session, we can use Python scripts by specifying the service url and the api\_key. The following lines of code return the url, the api\_key, the help\_url, and the service\_id. 

In [7]:
service_url = demoservice.service.url 
api_key = demoservice.service.api_key
help_url = demoservice.service.help_url
service_id = demoservice.service.service_id

The purpose of the following code is to wait a few seconds so that the web service is successfully desployed and ready for consumption.

In [8]:
# wait 
import time
time.sleep(10)

The code below can be used to consume the web service. For illustration purpose the data for two records are provided.

In [9]:
import urllib2
# If you are using Python 3+, import urllib instead of urllib2

import json 

data =  {

        "Inputs": {

                "input1":
                {
                    "ColumnNames": ["crim", "zn", "lstat", "age", "tax", "rad", "black", 
                                    "chas", "nox", "rm", "indus", "ptratio", "dis"],
                    "Values": [ [ "0.00632", "18", "4.98", "65.2", "296", "1", "396.9", 
                                 "0", "0.538", "6.575", "2.31", "15.3", "4.09" ], 
                               ["0.02731", "0", "9.14", "78.9", "242", "2", "396.9", 
                                "0", "0.469", "6.421", "7.07", "17.8", "4.9671"],
                            ]
                },        },
            "GlobalParameters": {
}
    }

body = str.encode(json.dumps(data))

url = service_url
api_key = api_key # Replace this with the API key for the web service

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib2.Request(url, body, headers) 

try:
    response = urllib2.urlopen(req)

    # If you are using Python 3+, replace urllib2 with urllib.request in the above code:
    # req = urllib.request.Request(url, body, headers) 
    # response = urllib.request.urlopen(req)

    result = response.read()
    print(result) 
except urllib2.HTTPError, error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, 
    # which are useful for debugging the failure
    print(error.info())

    print(json.loads(error.read()))                 

{"Results":{"output1":{"type":"table","value":{"Values":[["30.0082126923446"],["25.0298606038188"]]}},"output2":{"type":"table","value":{"Values":[["data:text/plain,","data:text/plain,",null]]}}}}


The above service consumption code is based on those at the help\_url, which contains, among others, sample code written in C#, Python, and R for consuming the web service. To check out the code for yourself, copy the help\_url value to a new browser to open the web page. Scroll down the newly opened page till you see the section "Sample Code" as in [Figure 1][pic 1]. Click on the Python tab and you'll see the sample code. Two changes were made in the above code:
    1. fill in the url and api\_key with the values returned for the current web service, and 
    2. enter the values for the first two records. Notice that the order of the columns here may be different from 
    that in your definition of the web service. 

[![Python Script][pic 1]][pic 1] Figure 1

[pic 1]: https://az712634.vo.msecnd.net/notebooks/d132b658-c5bd-11e5-97a3-0ba617781c98.PNG

### 4.3 Re-publish a web service
By default, when you run the code in section 4.1 multiple times different services will be generated. They will have different url, api\_key, help\_url, and service\_id. If you just want to re-publish the same service, you need to specify the service\_id as shown below. This way, you can use the same code for consumption without having to change the url and api\_key.

In [10]:
from azureml import services
@services.publish(workdspace_id, authorization_token)
@services.service_id(service_id)
@services.types(crim=float, zn=float, indus=float, chas=float, nox=float, rm=float, age=float, 
                dis=float, rad=float, tax=float, ptratio=float, black=float, lstat=float)
@services.returns(float)
def demoservice(crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, black, lstat):
    # predict the label
    feature_vector = [crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, black, lstat]
    return lm.predict(feature_vector)

## 5 Conclusion
Through this example, you've learned how to fit a model, deploy the model on Azure, and consume the service, all using Azure ML's Jupyter notebook. 

In addition to carrying out the entire model development process with Azure ML notebook, you can also use it as a supplement to Azure ML's experiments. Using Azure ML experiments you can develop a wide variety of models and easily compare their performance. However, certain tasks may not be accomplished there. Examples include certain feature selection techniques, advanced visualization options, a wide variety of GBM models, and time series analysis. For these tasks, Python offers a good alternative and Azure ML's Jupyter notebook allows you to write and run Python programs on the cloud.

As another example of using Jupyter noteebook, you can test out Python code that will be used in the "Execute Python Script" module in an Azure ML Studio experiment.

---  
Created by a Microsoft Employee.  
Copyright (C) Microsoft. All Rights Reserved.